In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import contextily as ctx
# from shapely.geometry import Point, Polygon
# from pyproj import CRS
# from sklearn.linear_model import LinearRegression
# import seaborn as sns
# import statsmodels.api as sm
import folium
from folium.plugins import MarkerCluster, FastMarkerCluster, HeatMap#, heat_map_withtime, HeatMapWithTime
from folium.plugins.heat_map_withtime import HeatMapWithTime
from sklearn.neighbors import KernelDensity
import branca.colormap as cm

In [2]:
import zipfile

zip_file_path = 'Data/Wildland_Fire_Incident_Locations.csv.zip'

# Open the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_file:
    df = pd.read_csv(zip_file.open('Wildland_Fire_Incident_Locations.csv'))

# X coordinate (longitude)
# Y coordinate (latitude)

/Users/keanan/opt/anaconda3/envs/flatiron-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (6,7,15,16,34,63,66,70,71,79,87,88,92,93) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [131]:
# only using keep1, dates and target
keep = ['IncidentSize', 'EstimatedCostToDate', 'FinalAcres', 'FireCause', 'PrimaryFuelModel',
        'EstimatedFinalCost', 'IncidentName']

maybe = ['FireCauseGeneral', 'FireCauseSpecific', 'DiscoveryAcres', 'FireMgmtComplexity']

keep1 = ['X','Y', 'OBJECTID','FireDiscoveryDateTime', 'FireOutDateTime', 'FireMgmtComplexity']

keep3 = ['DispatchCenterID','POOJurisdictionalAgency']

df['FireOutDateTime'].fillna(df['ContainmentDateTime'], inplace=True)
df['DispatchCenterID'].fillna(df['POODispatchCenterID'], inplace=True)

df1 = df[keep1]
df1.dropna(inplace=True)
df1

<ipython-input-131-0e3afa8e6581>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.dropna(inplace=True)


,X,Y,OBJECTID,FireDiscoveryDateTime,FireOutDateTime,FireMgmtComplexity
982,-93.772647,29.861605,983,2022/12/10 01:09:18+00,2022/12/11 19:30:00+00,Type 4 Incident
1817,-121.604615,47.638906,1818,2022/09/02 20:08:00+00,2022/12/16 21:13:00+00,Type 4 Incident
4407,-121.517215,46.635556,4408,2022/08/09 16:43:00+00,2023/04/26 19:52:00+00,Type 5 Incident
13105,-117.058714,48.648346,13108,2022/07/14 22:28:59+00,2023/01/03 20:00:00+00,Type 4 Incident
52351,-122.050314,43.363345,52366,2022/07/30 23:16:59+00,2023/02/01 18:00:00+00,Type 3 Incident
...,...,...,...,...,...,...
257441,-88.356705,35.919319,307005,2023/05/04 14:49:20+00,2023/05/05 14:47:03+00,Type 1 Prescribed Fire
257469,-88.868006,35.767107,307037,2023/05/04 16:00:01+00,2023/05/04 17:53:02+00,Type 1 Prescribed Fire
257487,-83.263634,37.351947,307059,2023/05/04 16:05:59+00,2023/05/04 17:22:27+00,Type 5 Incident
257533,-85.286071,35.959857,307115,2023/05/04 21:20:59+00,2023/05/04 22:35:05+00,Type 5 Incident


In [132]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257625 entries, 0 to 257624
Data columns (total 96 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   X                                257625 non-null  float64
 1   Y                                257625 non-null  float64
 2   OBJECTID                         257625 non-null  int64  
 3   SourceOID                        257625 non-null  int64  
 4   ABCDMisc                         13081 non-null   object 
 5   ADSPermissionState               257625 non-null  object 
 6   ContainmentDateTime              154829 non-null  object 
 7   ControlDateTime                  139224 non-null  object 
 8   CreatedBySystem                  257625 non-null  object 
 9   IncidentSize                     180988 non-null  float64
 10  DiscoveryAcres                   192242 non-null  float64
 11  DispatchCenterID                 225143 non-null  object 
 12  Es

In [133]:
# set dates to date time
def datetimes(df, dates=None, normal=True):
    '''
    df : dataframe
    dates : list of columns to perform datetime on, if none only columns with Date in name is used
    normal : True, False will not use format='%Y-%m-%d'
    '''
    if not dates:
        dates = list(df.columns[df.columns.str.contains('Date')])
    if normal:
        for t in dates:
            df[t] = pd.to_datetime(df[t], errors='coerce', format='%Y-%m-%d')
            df[t] = df[t].dt.tz_localize(None)
            print('Used Format Y-m-d')
    else:
        for t in dates:
            df[t] = pd.to_datetime(df[t], errors='coerce')
            
datetimes(df1)

Used Format Y-m-d
Used Format Y-m-d


<ipython-input-133-016400175305>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[t] = pd.to_datetime(df[t], errors='coerce', format='%Y-%m-%d')
<ipython-input-133-016400175305>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[t] = df[t].dt.tz_localize(None)


In [134]:
df1.isna().sum()

X                        0
Y                        0
OBJECTID                 0
FireDiscoveryDateTime    0
FireOutDateTime          0
FireMgmtComplexity       0
dtype: int64

In [135]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17546 entries, 982 to 257534
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   X                      17546 non-null  float64       
 1   Y                      17546 non-null  float64       
 2   OBJECTID               17546 non-null  int64         
 3   FireDiscoveryDateTime  17546 non-null  datetime64[ns]
 4   FireOutDateTime        17546 non-null  datetime64[ns]
 5   FireMgmtComplexity     17546 non-null  object        
dtypes: datetime64[ns](2), float64(2), int64(1), object(1)
memory usage: 959.5+ KB


In [136]:
df1.head()

,X,Y,OBJECTID,FireDiscoveryDateTime,FireOutDateTime,FireMgmtComplexity
982,-93.772647,29.861605,983,2022-12-10 01:09:18,2022-12-11 19:30:00,Type 4 Incident
1817,-121.604615,47.638906,1818,2022-09-02 20:08:00,2022-12-16 21:13:00,Type 4 Incident
4407,-121.517215,46.635556,4408,2022-08-09 16:43:00,2023-04-26 19:52:00,Type 5 Incident
13105,-117.058714,48.648346,13108,2022-07-14 22:28:59,2023-01-03 20:00:00,Type 4 Incident
52351,-122.050314,43.363345,52366,2022-07-30 23:16:59,2023-02-01 18:00:00,Type 3 Incident


In [305]:
# only use fires in the contiguous usa
def conti(df):   
    '''
    This function assumes lat and long are X and Y
    limits dataset to only contiguous USA
    '''
    max_lat, min_lat = 49.384358, 24.396308
    max_long, min_long = -66.934570, -125.000000
    
    site = df.copy()

    site['Y'] = site['Y'].apply(lambda y: np.nan if y > max_lat or y < min_lat else y)
    site['X'] = site['X'].apply(lambda x: np.nan if x > max_long or x < min_long else x)
    site.dropna(subset=['X', 'Y'], inplace =True)
    return site
df22 = conti(df)
len(df22)


252653

In [273]:
# further cleanin
def fixer(df):
    '''
    Function to further clean fire occurences dataset
    '''
    # remove any year thats older then 2014 (dataset is only supposed to go back to 2014)
    datetime_columns = list(df.select_dtypes(['datetime64[ns, UTC]', 'datetime64[ns]']).columns)
    
    # Replace missing FireOutDateTime values with ContainmentDateTime values
    dropped = []
    for dc in datetime_columns:
        df['year'] = df[dc].dt.year
        dropped.extend(df[df['year'] < 2014].values)
        df = df[df['year'] >= 2014]
#         dropped.extend(df[df['year'] < 2014].index)
    df.drop(columns='year', inplace=True)
      
        
    # Drop any fire out day that is after the current date, as thats impossible 
    from datetime import datetime
    # Todays date
    today = datetime.now().date()

    # Drop rows where FireOutDateTime is greater then today
    dropped.extend(df[df['FireOutDateTime'].dt.date > today].values)
    df = df[df['FireOutDateTime'].dt.date <= today]
    
    # Drop rows where FireOutDateTime is greater then FireDiscoveryDateTime
    dropped.extend(df[df['FireOutDateTime'].dt.date >= df['FireDiscoveryDateTime'].dt.date].values)
    df = df[df['FireOutDateTime'].dt.date >= df['FireDiscoveryDateTime'].dt.date]
    
    return df, dropped

df2, dropped_rows = fixer(df2)


In [274]:
df2

,X,Y,OBJECTID,FireDiscoveryDateTime,FireOutDateTime,FireMgmtComplexity
982,-93.772647,29.861605,983,2022-12-10 01:09:18,2022-12-11 19:30:00,Type 4 Incident
1817,-121.604615,47.638906,1818,2022-09-02 20:08:00,2022-12-16 21:13:00,Type 4 Incident
4407,-121.517215,46.635556,4408,2022-08-09 16:43:00,2023-04-26 19:52:00,Type 5 Incident
13105,-117.058714,48.648346,13108,2022-07-14 22:28:59,2023-01-03 20:00:00,Type 4 Incident
52351,-122.050314,43.363345,52366,2022-07-30 23:16:59,2023-02-01 18:00:00,Type 3 Incident
...,...,...,...,...,...,...
257441,-88.356705,35.919319,307005,2023-05-04 14:49:20,2023-05-05 14:47:03,Type 1 Prescribed Fire
257469,-88.868006,35.767107,307037,2023-05-04 16:00:01,2023-05-04 17:53:02,Type 1 Prescribed Fire
257487,-83.263634,37.351947,307059,2023-05-04 16:05:59,2023-05-04 17:22:27,Type 5 Incident
257533,-85.286071,35.959857,307115,2023-05-04 21:20:59,2023-05-04 22:35:05,Type 5 Incident


In [276]:
# drop nulls
df2.dropna(inplace=True)

In [277]:
# Create a new column date with each day between
df2['date'] = df2.apply(lambda row: pd.date_range(row['FireDiscoveryDateTime'], row['FireOutDateTime'], freq='D'), axis=1)

# Explode the date column to create a row for each day
df2 = df2.explode('date')

# Drop the original original date columns
df2 = df2.drop(['FireDiscoveryDateTime', 'FireOutDateTime'], axis=1)

In [282]:
df2['year'] = df2['date'].dt.year
df2['month'] = df2['date'].dt.month
df2['date2'] = df2.apply(lambda row: f"{row['year']}-{row['month']}", axis=1)
df2 = df2.drop(['year', 'month'], axis=1)

In [280]:
df2

,X,Y,OBJECTID,FireMgmtComplexity,date,date2
982,-93.772647,29.861605,983,Type 4 Incident,2022-12-10 01:09:18,2022.0-12.0
982,-93.772647,29.861605,983,Type 4 Incident,2022-12-11 01:09:18,2022.0-12.0
1817,-121.604615,47.638906,1818,Type 4 Incident,2022-09-02 20:08:00,2022.0-9.0
1817,-121.604615,47.638906,1818,Type 4 Incident,2022-09-03 20:08:00,2022.0-9.0
1817,-121.604615,47.638906,1818,Type 4 Incident,2022-09-04 20:08:00,2022.0-9.0
...,...,...,...,...,...,...
257441,-88.356705,35.919319,307005,Type 1 Prescribed Fire,2023-05-04 14:49:20,2023.0-5.0
257469,-88.868006,35.767107,307037,Type 1 Prescribed Fire,2023-05-04 16:00:01,2023.0-5.0
257487,-83.263634,37.351947,307059,Type 5 Incident,2023-05-04 16:05:59,2023.0-5.0
257533,-85.286071,35.959857,307115,Type 5 Incident,2023-05-04 21:20:59,2023.0-5.0


In [283]:
# dropping rows so its on a monthly scale, only use if heat map with time is in months, if it is in days skip
df2 = df2.groupby('OBJECTID').apply(lambda x: x.drop_duplicates(subset='date2'))
df2 = df2.reset_index(drop=True)


In [285]:
# drop date 2
df2 = df2.drop(['date2'], axis=1)
df2

,X,Y,OBJECTID,FireMgmtComplexity,date
0,-93.772647,29.861605,983,Type 4 Incident,2022-12-10 01:09:18
1,-121.604615,47.638906,1818,Type 4 Incident,2022-09-02 20:08:00
2,-121.604615,47.638906,1818,Type 4 Incident,2022-10-01 20:08:00
3,-121.604615,47.638906,1818,Type 4 Incident,2022-11-01 20:08:00
4,-121.604615,47.638906,1818,Type 4 Incident,2022-12-01 20:08:00
...,...,...,...,...,...
23494,-88.356705,35.919319,307005,Type 1 Prescribed Fire,2023-05-04 14:49:20
23495,-88.868006,35.767107,307037,Type 1 Prescribed Fire,2023-05-04 16:00:01
23496,-83.263634,37.351947,307059,Type 5 Incident,2023-05-04 16:05:59
23497,-85.286071,35.959857,307115,Type 5 Incident,2023-05-04 21:20:59


In [286]:
# removing prescribed incidents
df2 = df2[df2['FireMgmtComplexity'] != 'Type 1 Prescribed Fire']
df2 = df2[df2['FireMgmtComplexity'] != 'Type 3 Prescribed Fire']

df2

,X,Y,OBJECTID,FireMgmtComplexity,date
0,-93.772647,29.861605,983,Type 4 Incident,2022-12-10 01:09:18
1,-121.604615,47.638906,1818,Type 4 Incident,2022-09-02 20:08:00
2,-121.604615,47.638906,1818,Type 4 Incident,2022-10-01 20:08:00
3,-121.604615,47.638906,1818,Type 4 Incident,2022-11-01 20:08:00
4,-121.604615,47.638906,1818,Type 4 Incident,2022-12-01 20:08:00
...,...,...,...,...,...
23492,-83.289337,36.191124,306874,Type 5 Incident,2023-05-03 18:09:26
23493,-85.712988,36.126074,306878,Type 5 Incident,2023-05-03 20:19:00
23496,-83.263634,37.351947,307059,Type 5 Incident,2023-05-04 16:05:59
23497,-85.286071,35.959857,307115,Type 5 Incident,2023-05-04 21:20:59


In [159]:
# df2['date'] = df2['date'].dt.month

In [287]:
df2['FireMgmtComplexity'].value_counts()

Type 5 Incident    10744
Type 4 Incident     8481
Type 3 Incident     3181
Type 2 Incident      630
Type 1 Incident      345
Name: FireMgmtComplexity, dtype: int64

In [289]:
df2

,X,Y,OBJECTID,FireMgmtComplexity,date
0,-93.772647,29.861605,983,Type 4 Incident,2022-12-10 01:09:18
1,-121.604615,47.638906,1818,Type 4 Incident,2022-09-02 20:08:00
2,-121.604615,47.638906,1818,Type 4 Incident,2022-10-01 20:08:00
3,-121.604615,47.638906,1818,Type 4 Incident,2022-11-01 20:08:00
4,-121.604615,47.638906,1818,Type 4 Incident,2022-12-01 20:08:00
...,...,...,...,...,...
23492,-83.289337,36.191124,306874,Type 5 Incident,2023-05-03 18:09:26
23493,-85.712988,36.126074,306878,Type 5 Incident,2023-05-03 20:19:00
23496,-83.263634,37.351947,307059,Type 5 Incident,2023-05-04 16:05:59
23497,-85.286071,35.959857,307115,Type 5 Incident,2023-05-04 21:20:59


In [290]:
# flipping for code purposes now type 1 is 5 to represent greater weight
df2['FireMgmtComplexity'] = df2['FireMgmtComplexity'].map({'Type 4 Incident': 2, 'Type 5 Incident': 1, 'Type 3 Incident': 3, 'Type 2 Incident': 4, 'Type 1 Incident': 5})
# df2['FireMgmtComplexity'] = df2['FireMgmtComplexity'].map({'Type 4 Incident': 4, 'Type 5 Incident': 5, 'Type 3 Incident': 3, 'Type 2 Incident': 2, 'Type 1 Incident': 1})


In [301]:
def heatmapwithtime(df,column, date_column,*args,**kwargs):
    import inspect
    '''
    df : DataFrame
    column :  weight_column
    date_column : date column w/ year,month, day
    *args : 'y', 'm', and/or 'd', order matters, or can be a metric 'mean', 'median', 'mode'
    **kwargs : parameters for folium.Map and HeatMapWithTime - blur only works with newer folium versions
    '''
    # creating a date-time column
    df[date_column] = pd.to_datetime(df[date_column], format='%m/%d/%Y')
    datetime = {'y': '%Y', 'm': '%m', 'd': '%d'}
    date_format = '-'.join(datetime[d] for d in args if d in datetime)
    df['datetime'] = df[date_column].dt.strftime(date_format)
    
    # scaling the column weight
    weight_min, weight_max = df[column].min(), df[column].max()
    metrics = {'mean': df[column].mean(), 'median': df[column].median(), 'mode': df[column].mode()} 
    # Could also be [metrics[m] for m in args if m in metrics] 
    # HOWEVER below metric needs to be metric[0] 
    metric = next((metrics[m] for m in args if m in metrics),None)
    if metric:
        df[column] = df[column].apply(lambda x: (x) / (metric))
    else:
        df[column] = df[column].apply(lambda x: (x - weight_min) / (weight_max - weight_min))
    
    # Data nested list
    data_np = np.vstack([df['Y'], df['X'], df[column]]).T
    data_nps = data_np.tolist()
    
    # Time Index list
    time_index_df = df['datetime']
    time_index_input = time_index_df.values.tolist()

    # final list
    data_list = list(zip(data_nps,time_index_input))
    f_df = pd.DataFrame(data_list)
    final_df = f_df.groupby(by=1).agg(lambda x: x.tolist())
    
    # creating input data for heatmap with time
    df_data = final_df[0].tolist()
    df_time = final_df.index.tolist()
    
    # Creating Folium base map
    folium_map = folium.Map(location=[df['Y'].mean(), df['X'].mean()], zoom_start=5,**kwargs)
    
    # Creating **params - this allows it to function similar to HeatMap, can take any **kwargs 
    map_param = inspect.getfullargspec(HeatMapWithTime).args[1:]
    param = {key: value for key,value in kwargs.items() if key in map_param}
    
    # Heat map with time
    HeatMapWithTime(df_data,index=df_time, auto_play=True,
                         min_speed=.5,speed_step=0.1, **param).add_to(folium_map)
    
#     folium_map.save('images/Heat_map.html')
    # Display Map
    display(folium_map)

In [298]:
# map tile types
dark_map = 'cartodbdark_matter'
stamentoner = "stamentoner"
litestamentoner = 'stamentonerlite'
light_map = 'cartodbpositron'
terrain = 'StamenTerrain'
ESRI_DeLorme = 'ESRIDeLorme'
ESRI_WorldImagery ='ESRIWorldImagery'
ESRI_NatGeoWorldMap = 'ESRINatGeoWorldMap'

In [299]:
# df2['date'] = pd.to_datetime(df2['date'])


In [300]:
# write code to make args optional 
heatmapwithtime(df2[:10000],'FireMgmtComplexity', 'date', 'y', 'm',
                min_opacity=0, max_opacity=.6,radius=12, tiles=stamentoner, max_speed=10)


<ipython-input-297-412ad303dfce>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[date_column] = pd.to_datetime(df[date_column], format='%m/%d/%Y')
<ipython-input-297-412ad303dfce>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['datetime'] = df[date_column].dt.strftime(date_format)
<ipython-input-297-412ad303dfce>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [302]:
# point style
heatmapwithtime(df2[:10000],'FireMgmtComplexity', 'date', 'y', 'm',
                min_opacity=.3, max_opacity=9,radius=3, tiles=stamentoner, max_speed=10)


<ipython-input-301-89c11b0f135a>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[date_column] = pd.to_datetime(df[date_column], format='%m/%d/%Y')
<ipython-input-301-89c11b0f135a>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['datetime'] = df[date_column].dt.strftime(date_format)
<ipython-input-301-89c11b0f135a>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat